## 제목 : Pytorch Classification NN
## 작성자 : 김수
## 작성 일자 : 2022.02.09

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
torch.manual_seed(777)
num_epoch = 1000
Learning_Rate = 0.01

In [2]:
training_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/분류대회/Training_data.csv",delimiter=",")
test_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/분류대회/Test_Input.csv",delimiter=",")

In [7]:
cut_line = int(training_data.shape[0]*0.7)
x = training_data[:cut_line,:-4]
y = training_data[:cut_line,-4:]
val_x = training_data[cut_line:,:-4]
val_y = training_data[cut_line:,-4:]

test_x = test_data[:,:]

X = torch.tensor(x,dtype=torch.float32)
Y = torch.reshape(torch.tensor(y,dtype=torch.float32),[y.shape[0],y.shape[1]])
Y_not_one_hot_encoding = torch.argmax(Y,dim=1)

Val_X = torch.tensor(val_x,dtype=torch.float32)
Val_Y = torch.reshape(torch.tensor(val_y,dtype=torch.float32),[val_y.shape[0],val_y.shape[1]])
Val_Y_not_one_hot_encoding = torch.argmax(Val_Y,dim=1)

TEST = torch.tensor(test_x,dtype=torch.float32)

X_re = torch.reshape(torch.tensor(X,dtype=torch.float32),[X.shape[0],1,X.shape[1]])
Y_re = torch.reshape(torch.tensor(y,dtype=torch.long),[Y.shape[0],Y.shape[1]])
Val_X_re = torch.reshape(torch.tensor(Val_X,dtype=torch.float32),[Val_X.shape[0],1,Val_X.shape[1]])
Val_Y_re = torch.reshape(torch.tensor(Val_Y,dtype=torch.float32),[Val_Y.shape[0],1,Val_Y.shape[1]])

TEST_re = torch.reshape(torch.tensor(TEST,dtype=torch.float32),[TEST.shape[0],1,TEST.shape[1]])


print(X_re.shape)
print(Y.shape)
print(Val_X_re.shape)
print(Val_Y.shape)
print(TEST_re.shape)

torch.Size([98000, 1, 24])
torch.Size([98000, 4])
torch.Size([42000, 1, 24])
torch.Size([42000, 4])
torch.Size([60000, 1, 24])


C:\Users\user\AppData\Local\Temp/ipykernel_11380/1297795176.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_re = torch.reshape(torch.tensor(X,dtype=torch.float32),[X.shape[0],1,X.shape[1]])
C:\Users\user\AppData\Local\Temp/ipykernel_11380/1297795176.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Val_X_re = torch.reshape(torch.tensor(Val_X,dtype=torch.float32),[Val_X.shape[0],1,Val_X.shape[1]])
C:\Users\user\AppData\Local\Temp/ipykernel_11380/1297795176.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Val_Y_re = torch.resh

In [4]:
# Binary Crossentropy 손실 = loss(x,y) = BCE
# Binary Crossentropy 손실 = loss(x,y) = BCE

# [1, 1, 98000, 12] -> [1, 40, 97996, 11]
# [1, 40, 97996, 11] -> [1, 40, 97992, 10]
# [1, 40, 97992, 10] -> [1, 40, 97991, 9]
# [1, 40, 97991, 9] -> [1, 40, 97990, 8]
# [1, 40, 97989, 8] -> [1, 40, 97988, 8]

model = nn.Sequential(nn.Conv1d(1,40, kernel_size=6),nn.ReLU(), 
                      nn.Conv1d(40, 40, kernel_size=3), nn.ReLU(),
                      nn.Conv1d(40, 40, kernel_size=3), nn.ReLU(),
                      nn.Conv1d(40, 40, kernel_size=3), nn.ReLU(),
                      nn.Conv1d(40, 1, kernel_size=3), nn.ReLU(),
                      nn.Flatten(),
                      nn.Linear(11, 100), nn.ELU(), nn.Dropout(0.3),
                      nn.Linear(100, 50), nn.ELU(),nn.Dropout(0.3),
                      nn.Linear(50, 25), nn.ELU(),nn.Dropout(0.3),
                     nn.Linear(25, Y.shape[1]), nn.Softmax())
print(model)
optimizer = optim.Adam(model.parameters() ,lr = Learning_Rate)
Loss_fun = torch.nn.CrossEntropyLoss()
Loss = np.zeros((num_epoch,1))
Val_Loss = np.zeros((num_epoch,1))
Accuracy = np.zeros((num_epoch,1))
Val_Accuracy = np.zeros((num_epoch,1))
#학습
for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(X_re)
    loss = Loss_fun(output,Y)
    loss.backward()
    optimizer.step()
    # Accuracy Calculate
    output_not_one_hot_encoding=torch.argmax(output,dim=1)
    accuracy = torch.divide(torch.count_nonzero(torch.eq(output_not_one_hot_encoding,
                                                         Y_not_one_hot_encoding)),Y_not_one_hot_encoding.shape[0])
    with torch.no_grad():
        val_output = model(Val_X_re)
        val_loss = Loss_fun(val_output,Val_Y)
        val_output_not_one_hot_encoding=torch.argmax(val_output,dim=1)
        val_accuracy = torch.divide(torch.count_nonzero(torch.eq(val_output_not_one_hot_encoding,
                                                         Val_Y_not_one_hot_encoding)),Val_Y_not_one_hot_encoding.shape[0])
    Loss[i,0] = loss.data
    Accuracy[i,0] = accuracy
    Val_Loss[i,0] = val_loss.data
    Val_Accuracy[i,0] = val_accuracy
    if i % 10 == 0:
        print(i,"Loss : ",loss.data,"Accuracy : ",accuracy.data,"Val Loss : ",val_loss.data,"Val Accuracy : ",val_accuracy.data)
        

Sequential(
  (0): Conv1d(1, 40, kernel_size=(6,), stride=(1,))
  (1): ReLU()
  (2): Conv1d(40, 40, kernel_size=(3,), stride=(1,))
  (3): ReLU()
  (4): Conv1d(40, 40, kernel_size=(3,), stride=(1,))
  (5): ReLU()
  (6): Conv1d(40, 40, kernel_size=(3,), stride=(1,))
  (7): ReLU()
  (8): Conv1d(40, 1, kernel_size=(3,), stride=(1,))
  (9): ReLU()
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=11, out_features=100, bias=True)
  (12): ELU(alpha=1.0)
  (13): Dropout(p=0.3, inplace=False)
  (14): Linear(in_features=100, out_features=50, bias=True)
  (15): ELU(alpha=1.0)
  (16): Dropout(p=0.3, inplace=False)
  (17): Linear(in_features=50, out_features=25, bias=True)
  (18): ELU(alpha=1.0)
  (19): Dropout(p=0.3, inplace=False)
  (20): Linear(in_features=25, out_features=4, bias=True)
  (21): Softmax(dim=None)
)


C:\Users\user\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 Loss :  tensor(1.3869) Accuracy :  tensor(0.2511) Val Loss :  tensor(1.3845) Val Accuracy :  tensor(0.2678)
10 Loss :  tensor(1.1733) Accuracy :  tensor(0.5353) Val Loss :  tensor(1.1336) Val Accuracy :  tensor(0.5896)
20 Loss :  tensor(1.0400) Accuracy :  tensor(0.6920) Val Loss :  tensor(1.0268) Val Accuracy :  tensor(0.7079)
30 Loss :  tensor(0.9929) Accuracy :  tensor(0.7442) Val Loss :  tensor(0.9916) Val Accuracy :  tensor(0.7434)
40 Loss :  tensor(0.9586) Accuracy :  tensor(0.7801) Val Loss :  tensor(0.9515) Val Accuracy :  tensor(0.7881)
50 Loss :  tensor(0.9414) Accuracy :  tensor(0.7981) Val Loss :  tensor(0.9399) Val Accuracy :  tensor(0.7997)
60 Loss :  tensor(0.9291) Accuracy :  tensor(0.8107) Val Loss :  tensor(0.9299) Val Accuracy :  tensor(0.8105)
70 Loss :  tensor(0.9239) Accuracy :  tensor(0.8164) Val Loss :  tensor(0.9232) Val Accuracy :  tensor(0.8167)
80 Loss :  tensor(0.9160) Accuracy :  tensor(0.8245) Val Loss :  tensor(0.9186) Val Accuracy :  tensor(0.8226)
90

740 Loss :  tensor(0.8639) Accuracy :  tensor(0.8788) Val Loss :  tensor(0.8780) Val Accuracy :  tensor(0.8643)
750 Loss :  tensor(0.8679) Accuracy :  tensor(0.8750) Val Loss :  tensor(0.8779) Val Accuracy :  tensor(0.8645)
760 Loss :  tensor(0.8637) Accuracy :  tensor(0.8792) Val Loss :  tensor(0.8794) Val Accuracy :  tensor(0.8629)
770 Loss :  tensor(0.8664) Accuracy :  tensor(0.8762) Val Loss :  tensor(0.8776) Val Accuracy :  tensor(0.8651)
780 Loss :  tensor(0.8624) Accuracy :  tensor(0.8803) Val Loss :  tensor(0.8774) Val Accuracy :  tensor(0.8660)
790 Loss :  tensor(0.8633) Accuracy :  tensor(0.8795) Val Loss :  tensor(0.8780) Val Accuracy :  tensor(0.8647)
800 Loss :  tensor(0.8631) Accuracy :  tensor(0.8798) Val Loss :  tensor(0.8781) Val Accuracy :  tensor(0.8642)
810 Loss :  tensor(0.8629) Accuracy :  tensor(0.8799) Val Loss :  tensor(0.8792) Val Accuracy :  tensor(0.8636)
820 Loss :  tensor(0.8638) Accuracy :  tensor(0.8791) Val Loss :  tensor(0.8771) Val Accuracy :  tensor(

In [8]:
Predict = model(TEST_re)
Predict_not_one_hot_encoding = torch.argmax(Predict,dim=1)
print(Predict_not_one_hot_encoding)

tensor([2, 2, 0,  ..., 2, 1, 0])


In [10]:
np.savetxt("G:/내 드라이브/연구실/PyTorch/6주차/Test_Predict.csv",Predict_not_one_hot_encoding,delimiter=",",fmt='%d')